In [80]:
# !pip install folium
#!pip install utm
#!pip install shapely

In [81]:
from h3 import h3
import utm
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
path = r'C:\Users\NiFa\Desktop\Data Science\CapStone\data\Ride_Data_Cache'
order_data = pd.read_msgpack('/mnt/c/Ride_Data_Cache/order_20161102.msgpack')

In [136]:
gps_data = pd.read_msgpack('/mnt/c/Ride_Data_Cache/gps_20161102.msgpack')

In [137]:
drivers = gps_data[['driver_id', 'order_id']].drop_duplicates()
order_data  = order_data.merge(drivers, on ='order_id', how='left')

In [138]:
order_data.head()

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_zone,driver_id
0,39d471630d26439111a7746d164d34eb,1478091677,1478092890,104.008160,30.706220,104.064147,30.685848,90.0,af17fba73f06db61f7596163eac1eb73
1,6681082cbcc11b37842489a096e2172e,1478093722,1478094806,104.053993,30.684104,104.015670,30.633230,14.0,af17fba73f06db61f7596163eac1eb73
2,56728c3d9555a452307d47986a773d9d,1478081351,1478082003,104.068120,30.708790,104.049720,30.704970,79.0,ae8221abe491455c7c0bedb0f00d6e8a
3,5b2724a9fd290d0388264fe07a9aec4c,1478082542,1478083418,104.059534,30.696013,104.034630,30.720960,14.0,ae8221abe491455c7c0bedb0f00d6e8a
4,0a2055d9c863e2f1963856d4d305ccd9,1478078909,1478080807,104.079048,30.796888,104.083284,30.705019,65.0,ae8221abe491455c7c0bedb0f00d6e8a


In [19]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)   
from h3 import h3
import folium

def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

In [46]:
minlat = min(order_data.pickup_latitude)
minlong = min(order_data.pickup_longitude)
maxlat = max(order_data.pickup_latitude)
maxlong = max(order_data.pickup_longitude)

In [47]:
geoJson = {'type': 'Polygon',
  'coordinates': [[[minlat, minlong], [minlat, maxlong ], [maxlat, maxlong], [ maxlat, minlong ]]] }

In [146]:
# geoJson = {'type': 'Polygon',
#  'coordinates': [[[37.813318999983238, -122.4089866999972145], [ 37.7866302000007224, -122.3805436999997056 ], [37.7198061999978478, -122.3544736999993603], [ 37.7076131999975672, -122.5123436999983966 ], [37.7835871999971715, -122.5247187000021967],  [37.8151571999998453, -122.4798767000009008]]] }

polyline = geoJson['coordinates'][0]
polyline.append(polyline[0])
lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]
m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color="green")
m.add_child(my_PolyLine)

hexagons = list(h3.polyfill(geoJson, 6))
polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v:v[0],polyline))
    lng.extend(map(lambda v:v[1],polyline))
    polylines.append(polyline)
for polyline in polylines:
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color='red')
    m.add_child(my_PolyLine)
display(m)

In [66]:
polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
a= np.array(polygons[0][0])

In [73]:
b = utm.from_latlon(a[:,0],a[:,1])

In [106]:
xy_pickup = utm.from_latlon(order_data.pickup_latitude.values,order_data.pickup_longitude.values)
x_pickup = list(xy_pickup[0])
y_pickup = list(xy_pickup[1])
pickup_point = list(zip(x_pickup, y_pickup))


In [111]:
for i,hex in enumerate(hexagons):
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    a= np.array(polygons[0][0])
    b = utm.from_latlon(a[:,0],a[:,1])
    poly_hex[i] = list(zip(b[0], b[1])) 

In [128]:
pick_zone = np.zeros(len(order_data))-1

for j,p in enumerate(pickup_point):
    point = Point(p)
    for i in range(len(poly_hex)):
        polygon = Polygon(poly_hex[i])
        if polygon.contains(point):
            pick_zone[j] = int(i)
            break
    

order_data['pickup_zone'] = pick_zone

In [142]:
sum(pick_zone==-1)

5

In [141]:
order_data.head()

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_zone,driver_id
0,39d471630d26439111a7746d164d34eb,1478091677,1478092890,104.008160,30.706220,104.064147,30.685848,90.0,af17fba73f06db61f7596163eac1eb73
1,6681082cbcc11b37842489a096e2172e,1478093722,1478094806,104.053993,30.684104,104.015670,30.633230,14.0,af17fba73f06db61f7596163eac1eb73
2,56728c3d9555a452307d47986a773d9d,1478081351,1478082003,104.068120,30.708790,104.049720,30.704970,79.0,ae8221abe491455c7c0bedb0f00d6e8a
3,5b2724a9fd290d0388264fe07a9aec4c,1478082542,1478083418,104.059534,30.696013,104.034630,30.720960,14.0,ae8221abe491455c7c0bedb0f00d6e8a
4,0a2055d9c863e2f1963856d4d305ccd9,1478078909,1478080807,104.079048,30.796888,104.083284,30.705019,65.0,ae8221abe491455c7c0bedb0f00d6e8a


In [144]:
temp = order_data.groupby(['driver_id', 'pickup_zone']).count()/ order_data.groupby(['driver_id']).count()

In [145]:
temp

dropoff_latitude  \
driver_id                        pickup_zone                     
000203e2b9574dec0f9e22e1471d7343 58.0                 1.000000   
00020e3d875210ba7b86de9a17a969d9 14.0                 0.600000   
                                 143.0                0.400000   
0002a9da2932f75a1d5f12fc99ea9945 14.0                 0.600000   
                                 79.0                 0.100000   
                                 142.0                0.050000   
                                 143.0                0.250000   
0003bf704c5549109b8379e8d0dcb68d 14.0                 0.500000   
                                 142.0                0.250000   
                                 143.0                0.250000   
00058555e69de8e0873eb611696f34df 143.0                1.000000   
0005c42f45ff69728cafba368569597e 14.0                 0.642857   
                                 79.0                 0.071429   
                                 90.0                 0.071429   
                                 142.0                0.142857   
                                 179.0                0.071429   
000dd90b5394f0979904c9ae65202da3 14.0                 0.250000   
                                 79.0                 0.125000   
                                 90.0                 0.125000   
                                 132.0                0.125000   
                                 142.0                0.250000   
                                 143.0                0.125000   
00103baf80d3029bfce826b8547b9712 65.0                 1.000000   
0010f62eb80ddb2885d38edccfd96d8e 14.0                 0.653846   
                                 79.0                 0.038462   
                                 87.0                 0.038462   
                                 142.0                0.038462   
                                 143.0                0.230769   
0011076352bb1288753bb0ef0a6deb23 14.0                 1.000000   
0011740700ed95c9c04f1e9e7871c78b 14.0                 0.428571   
...                                                        ...   
ffeb4c7d516d2be1902298ceb1077b13 143.0                0.333333   
fff119bb5ea38c3c02c24b1da0d77e3c 90.0                 1.000000   
fff19f7a18430336180f1cf541a7b6e9 14.0                 0.750000   
                                 26.0                 0.250000   
fff1c81848e5b8f7c0ef3313ac272b7c 14.0                 0.444444   
                                 79.0                 0.111111   
                                 105.0                0.222222   
                                 143.0                0.111111   
                                 179.0                0.111111   
fff4d1689ca5ff701f3f6e640c0ca245 14.0                 0.500000   
                                 142.0                0.125000   
                                 143.0                0.375000   
fff5b7a9b968fd25d757bc1a8ef34651 14.0                 0.722222   
                                 77.0                 0.055556   
                                 142.0                0.055556   
                                 143.0                0.111111   
                                 151.0                0.055556   
fff7162fdbc7f2c303cb0b811ea11e99 14.0                 0.416667   
                                 79.0                 0.166667   
                                 90.0                 0.166667   
                                 143.0                0.166667   
                                 179.0                0.083333   
fffa68525ebba2ef2a8f242aed38049d 14.0                 0.500000   
                                 79.0                 0.125000   
                                 132.0                0.125000   
                                 143.0                0.250000   
fffdc327efefe4b72e42ffd91ff7b489 14.0                 0.571429   
                                 59.0                 0.142857   
                    